In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('파일럿PJ_2차.csv', encoding='utf-8-sig')

/var/folders/x2/c6gjhmqs6h5bl_d26ctv923h0000gn/T/ipykernel_82021/3425303855.py:1: DtypeWarning: Columns (9,10,11,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('파일럿PJ_2차.csv', encoding='utf-8-sig')


In [164]:
df.columns

Index(['Unnamed: 0', '매장코드', '매장명', '스타일코드', '스타일명', '최초입고일', '최종입고일', '최초출고일',
       '최종출고일', '현판가', '예판가', '결판가', '단품코드', '누적매장순수입고량', '누적입고량', '누적출고량',
       '누적판매량', '기간판매량', '잔여재고량', '누적판매율[입고량대비]', '기간판매율[입고량대비]', '기간입고량',
       '기간출고량', 'Date'],
      dtype='object')

In [156]:
code_list = df['스타일코드'].unique()

In [162]:
df[df['스타일코드'] == 'SPTAE23M91']

,Unnamed: 0,매장코드,매장명,스타일코드,스타일명,최초입고일,최종입고일,최초출고일,최종출고일,현판가,예판가,결판가,단품코드,누적매장순수입고량,누적입고량,누적출고량,누적판매량,기간판매량,잔여재고량,누적판매율[입고량대비],기간판매율[입고량대비],기간입고량,기간출고량,Date


In [159]:
print('최초출고일:', df[df['스타일코드'] == 'SPTAE23M91']['최초출고일'].iloc[0])

IndexError: single positional indexer is out-of-bounds

In [158]:
for i in ['SPTAE23M91']:
    print('-'*50)
    print(i)
    print(df[df['스타일코드'] == i]['스타일명'].iloc[0])
    print('최초출고일:', df[df['스타일코드'] == i]['최초출고일'].iloc[0])
    print('최초판매일:', df[df['스타일코드'] == i]['Date'].iloc[0])

--------------------------------------------------
SPTAE23M91


IndexError: single positional indexer is out-of-bounds

In [6]:
import pandas as pd
import os

# 엑셀 파일 경로 설정
excel_file_path = '고대PJ_RAW_2.xlsx'  # 파일 경로와 이름을 적절히 변경해주세요

# 엑셀 파일 읽기
xls = pd.ExcelFile(excel_file_path)

sheet_names = xls.sheet_names
sheet_names.remove('스타일 정보')

In [10]:
# 각 시트를 데이터프레임으로 읽어와 리스트에 저장
dfs = []

for sheet in sheet_names:
    df = pd.read_excel(excel_file_path, sheet_name=sheet)
    df = df.drop(0, axis=0).reset_index(drop=True)
    df['Date'] = sheet
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
merged_df = pd.concat(dfs, ignore_index=True)

In [12]:
merged_df.to_csv('파일럿PJ_2차.csv', encoding='utf-8-sig')

In [13]:
code = pd.read_excel('매장코드구분.xlsx')

In [14]:
valid_code = code[code['반영여부'] == '반영']

In [15]:
filtered_df = merged_df[merged_df['매장코드'].isin(valid_code['매장코드'])]

In [17]:
filtered_df.to_csv('파일럿PJ_분석매장반영_2차.csv', encoding='utf-8-sig')

In [18]:
df = filtered_df

In [ ]:
# SPTAEA9M01 (array(['[에센셜] 와이드 슬랙스'], dtype=object), '231220', 0.6079570853822083)
# SPTAEA9M02 (array(['[에센셜] 롱 테이퍼드 슬랙스'], dtype=object), '231221', 0.42659193523275724)
# SPTAE23M03 (array(['[에센셜] 크롭 테이퍼드 슬랙스 (-5CM)'], dtype=object), '231221', 0.09641873278236915)
# SPTAE23M04 (array(['[에센셜] 풀밴딩 테이퍼드 슬랙스'], dtype=object), '231221', 0.0)

In [20]:
code_list = ['SPTAEA9M01', 'SPTAEA9M02', 'SPTAE23M03', 'SPTAE23M04']

# 1. 결품률

In [153]:
def cal_minus(x):
    minus = len(df[(df['스타일코드'] == x) & (df['잔여재고량'] <= 0)]) / len(df[df['스타일코드'] == x]) * 100
    name = df[df['스타일코드']== x]['스타일명'].unique()
    period = df[df['스타일코드'] == x]['Date'].iloc[0]
    return x, name, period, minus

In [154]:
for i in code_list:
    print(cal_minus(i))

('SPTAEA9M01', array(['[에센셜] 와이드 슬랙스'], dtype=object), Timestamp('2023-12-20 00:00:00'), 0.4552741263225393)
('SPTAEA9M02', array(['[에센셜] 롱 테이퍼드 슬랙스'], dtype=object), Timestamp('2023-12-21 00:00:00'), 0.33079722130334105)
('SPTAE23M03', array(['[에센셜] 크롭 테이퍼드 슬랙스 (-5CM)'], dtype=object), Timestamp('2023-12-21 00:00:00'), 0.11730205278592376)
('SPTAE23M04', array(['[에센셜] 풀밴딩 테이퍼드 슬랙스'], dtype=object), Timestamp('2023-12-21 00:00:00'), 0.016291951775822745)


In [52]:
a = (0.4552741263225393+0.33079722130334105)/2
a

0.39303567381294013

In [53]:
b= (0.11730205278592376+0.016291951775822745)/2
b

0.06679700228087325

In [54]:
(a-b)/a

0.8300485000945121

# 2. 매장 소진율 편차

In [23]:
df['기간판매율[입고량대비]'] = pd.to_numeric(df['기간판매율[입고량대비]'], errors='coerce').astype(float)

/var/folders/x2/c6gjhmqs6h5bl_d26ctv923h0000gn/T/ipykernel_11387/2921087818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['기간판매율[입고량대비]'] = pd.to_numeric(df['기간판매율[입고량대비]'], errors='coerce').astype(float)


In [60]:
df['누적판매율[입고량대비]'] = pd.to_numeric(df['누적판매율[입고량대비]'], errors='coerce').astype(float)

/var/folders/x2/c6gjhmqs6h5bl_d26ctv923h0000gn/T/ipykernel_11387/3385534161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['누적판매율[입고량대비]'] = pd.to_numeric(df['누적판매율[입고량대비]'], errors='coerce').astype(float)


In [62]:
df2 = df[df['스타일코드'].isin(code_list)]

In [39]:
df3 = df2.dropna(subset='기간출고량')
df3.head()

,매장코드,매장명,스타일코드,스타일명,최초입고일,최종입고일,최초출고일,최종출고일,현판가,예판가,...,누적입고량,누적출고량,누적판매량,기간판매량,잔여재고량,누적판매율[입고량대비],기간판매율[입고량대비],기간입고량,기간출고량,Date
99516,ABR6,디큐브거제,SPTAE23M03,[에센셜] 크롭 테이퍼드 슬랙스 (-5CM),2023-11-22,2023-12-04,2023-12-11,2024-01-09,39900,39900,...,4,-1,NaN,NaN,3,NaN,NaN,NaN,-1,231226
184983,ACVZ,스타필드코엑스몰,SPTAE23M03,[에센셜] 크롭 테이퍼드 슬랙스 (-5CM),2023-11-22,2023-12-04,2023-12-11,2024-01-09,39900,39900,...,6,-1,NaN,NaN,5,NaN,NaN,NaN,-1,240102
184991,ACVZ,스타필드코엑스몰,SPTAE23M04,[에센셜] 풀밴딩 테이퍼드 슬랙스,2023-11-07,2023-11-07,2023-11-09,2024-01-09,39900,39900,...,12,-1,2,NaN,9,18.181818,NaN,NaN,-1,240102
377161,ACVZ,스타필드코엑스몰,SPTAE23M03,[에센셜] 크롭 테이퍼드 슬랙스 (-5CM),2023-11-22,2023-12-04,2023-12-11,2024-01-09,39900,39900,...,24,-1,2,1,21,8.695652,4.347826,NaN,-1,240114
409184,A9WP,강남,SPTAEA9M01,[에센셜] 와이드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-15,39900,39900,...,18,-1,1,NaN,16,5.882353,NaN,NaN,-1,240116


In [130]:
df2_fillna = df2.fillna(0)

In [148]:
df_mean = df2.groupby(['매장코드', '매장명', '스타일코드', 'Date'])[['기간판매율[입고량대비]', '누적판매율[입고량대비]']].mean()

In [149]:
df_mean = df_mean.reset_index()

In [150]:
df_mean.head()

,매장코드,매장명,스타일코드,Date,기간판매율[입고량대비],누적판매율[입고량대비]
0,A8LC,2001중계,SPTAE23M03,2023-12-21,NaN,NaN
1,A8LC,2001중계,SPTAE23M03,2023-12-22,NaN,NaN
2,A8LC,2001중계,SPTAE23M03,2023-12-23,NaN,NaN
3,A8LC,2001중계,SPTAE23M03,2023-12-24,NaN,NaN
4,A8LC,2001중계,SPTAE23M03,2023-12-25,NaN,NaN


In [134]:
df_mean = df2.groupby(['매장코드', '매장명', '스타일코드', 'Date'])[['기간판매율[입고량대비]', '누적판매율[입고량대비]']].mean()

In [152]:
df_mean.head()

,매장코드,매장명,스타일코드,Date,기간판매율[입고량대비],누적판매율[입고량대비]
0,A8LC,2001중계,SPTAE23M03,2023-12-21,NaN,NaN
1,A8LC,2001중계,SPTAE23M03,2023-12-22,NaN,NaN
2,A8LC,2001중계,SPTAE23M03,2023-12-23,NaN,NaN
3,A8LC,2001중계,SPTAE23M03,2023-12-24,NaN,NaN
4,A8LC,2001중계,SPTAE23M03,2023-12-25,NaN,NaN


In [135]:
df_mean2 = df_mean.groupby(['스타일코드', 'Date'])['기간판매율[입고량대비]'].std()
df_mean2.groupby('스타일코드').mean()

스타일코드
SPTAE23M03    14.379853
SPTAE23M04    10.589850
SPTAEA9M01    14.619301
SPTAEA9M02    15.602671
Name: 기간판매율[입고량대비], dtype: float64

In [145]:
a = (14.619301 + 15.602671)/2
a

15.110986

In [146]:
b = (14.379853+10.589850)/2
b

12.484851500000001

In [147]:
(a-b)/a

0.17378975137691208

In [139]:
df_mean = df_mean.reset_index()

In [143]:
df_mean[df_mean['Date']=='2024-01-11'].groupby('스타일코드')['누적판매율[입고량대비]'].std()

스타일코드
SPTAE23M03     9.831807
SPTAE23M04     7.602375
SPTAEA9M01     7.712535
SPTAEA9M02    11.070773
Name: 누적판매율[입고량대비], dtype: float64

In [144]:
df_mean[df_mean['Date']=='2024-01-20'].groupby('스타일코드')['누적판매율[입고량대비]'].std()

스타일코드
SPTAE23M03    9.851742
SPTAE23M04    7.942056
SPTAEA9M01    8.194960
SPTAEA9M02    9.004272
Name: 누적판매율[입고량대비], dtype: float64

In [95]:
df_mean[(df_mean['Date'] == '2024-01-20') & (df_mean['스타일코드']=='SPTAEA9M02')].to_excel('데이터살펴보기.xlsx')

In [86]:
a = (14.619301+15.602671)/2
a

15.110986

In [87]:
b = (14.379853+10.589850)/2
b

12.484851500000001

In [88]:
(a-b)/a

0.17378975137691208

In [97]:
pd.set_option('display.max_columns',None)

In [98]:
df[(df['매장코드'] == 'A8MB')&(df['스타일코드']=='SPTAEA9M02')&(df['Date']=='2024-01-20')]

,매장코드,매장명,스타일코드,스타일명,최초입고일,최종입고일,최초출고일,최종출고일,현판가,예판가,결판가,단품코드,누적매장순수입고량,누적입고량,누적출고량,누적판매량,기간판매량,잔여재고량,누적판매율[입고량대비],기간판매율[입고량대비],기간입고량,기간출고량,Date,Week
483982,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0216074,3,3,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,2024-01-20,0
483983,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0216078,5,5,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,2024-01-20,0
483984,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0216082,7,7,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,2024-01-20,0
483985,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0216086,6,6,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,2024-01-20,0
483986,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0216090,3,3,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,2024-01-20,0
483987,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0217074,3,3,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,2024-01-20,0
483988,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0217078,5,5,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,2024-01-20,0
483989,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0217082,7,7,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,2024-01-20,0
483990,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0217086,6,6,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,2024-01-20,0
483991,A8MB,NC인천,SPTAEA9M02,[에센셜] 롱 테이퍼드 슬랙스,2023-12-04,2023-12-04,2023-12-11,2024-01-17,39900,39900,39900,SPTAEA9M0217090,3,3,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,2024-01-20,0


# 3. Week 별로

In [43]:
import pandas as pd

# Assuming 'Date' is in datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Create a new column 'Week' and initialize it with 0
df['Week'] = 0

# Assign week numbers based on date ranges
df.loc[(df['Date'] >= '2023-12-21') & (df['Date'] <= '2023-12-24'), 'Week'] = 1
df.loc[(df['Date'] >= '2023-12-25') & (df['Date'] <= '2023-12-31'), 'Week'] = 2
df.loc[(df['Date'] >= '2024-01-01') & (df['Date'] <= '2024-01-07'), 'Week'] = 3
df.loc[(df['Date'] >= '2024-01-08') & (df['Date'] <= '2024-01-11'), 'Week'] = 4
df.loc[(df['Date'] >= '2024-01-09') & (df['Date'] <= '2024-01-19'), 'Week'] = 5

/var/folders/x2/c6gjhmqs6h5bl_d26ctv923h0000gn/T/ipykernel_11387/3670749371.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')
/var/folders/x2/c6gjhmqs6h5bl_d26ctv923h0000gn/T/ipykernel_11387/3670749371.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week'] = 0


In [44]:
df2 = df[(df['Week'] != 0)]
df_mean = df2.groupby(['매장코드', '매장명', '스타일코드', 'Week'])['기간판매율[입고량대비]'].mean()
df_mean = df_mean.reset_index()

In [45]:
df_mean2 = df_mean[df_mean['스타일코드'].isin(code_list)]
df_mean2 = df_mean2.groupby(['스타일코드', 'Week'])['기간판매율[입고량대비]'].std()

In [46]:
df_mean2.groupby('스타일코드').mean()

스타일코드
SPTAE23M03    14.472696
SPTAE23M04     9.450116
SPTAEA9M01    13.678196
SPTAEA9M02    14.640147
Name: 기간판매율[입고량대비], dtype: float64

In [47]:
(13.678196+14.640147)/2

14.1591715

In [48]:
(14.472696+9.450116)/2

11.961406

In [ ]:
df_mean2.groupby(['스타일코드', 'Date'])['기간판매율[입고량대비]'].std().to_csv('기간판매율 편차.csv', encoding='utf-8-sig')